Build a recurrent neural network (LSTM) with TensorFlow.

References:
*   Long Short Term Memory, Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.




    
    


In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

W0822 10:13:38.147730 140123294599040 deprecation.py:323] From <ipython-input-1-9c1f5874b058>:7: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0822 10:13:38.149149 140123294599040 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0822 10:13:38.156840 140123294599040 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting /tmp/data/train-images-idx3-ubyte.gz


W0822 10:13:38.467133 140123294599040 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0822 10:13:38.471858 140123294599040 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0822 10:13:38.536847 140123294599040 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a f

Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# training hyper-parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

#network hyper-parameters
num_input = 28
timesteps = 28
num_hidden=128
num_classes=10

# graph input
X=tf.placeholder("float", [None, timesteps, num_input])
Y=tf.placeholder("float", [None, num_classes])

In [0]:
# define weights
weights={
    'out':tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [0]:
def RNN(x, weights, biases):
  # prepare data shape to match rnn function requirements
  # current data input shape: (batch, timesteps, n_iniput)
  # required shape: 'timesteps' tensorsl list of shape (batch, n_input)
  
  # unstack to get a list of 'timesteps' tensors of shape( batch, n_input)
  x = tf.unstack(x, timesteps, 1)
  
  # define lstm cell with tensorflow
  lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
  
  #get lstm cell output
  outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
  
  # linear activation, using rnn inner loop last output
  return tf.matmul(outputs[-1], weights['out'])+biases['out']

In [5]:
logits = RNN(X, weights,biases)
prediction = tf.nn.softmax(logits)

# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
logits=logits, labels=Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

# evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init=tf.global_variables_initializer()


W0822 10:13:38.846996 140123294599040 deprecation.py:323] From <ipython-input-4-b2cf4934da76>:10: __init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0822 10:13:38.849179 140123294599040 deprecation.py:323] From <ipython-input-4-b2cf4934da76>:13: static_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
W0822 10:13:38.880304 140123294599040 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing i

In [7]:
with tf.Session() as sess:
  sess.run(init)
  
  for step in range(1, training_steps+1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    
    #reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, timesteps, num_input))
    #run optimization op (backprop)
    sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
    
    if step%display_step ==0 or step ==1:
      loss, acc = sess.run([loss_op, accuracy], feed_dict = {X:batch_x, Y:batch_y})
      
      print("Step", str(step), "Loss", loss, "Accuracy", acc)
      
  print("Optimization finished!")
  
  test_len=128
  test_data=mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
  test_label=mnist.test.labels[:test_len]
  print("Testing accuracy:",sess.run(accuracy, feed_dict={X:test_data, Y:test_label}))
    

Step 1 Loss 2.5205305 Accuracy 0.078125
Step 200 Loss 2.1928666 Accuracy 0.25
Step 400 Loss 2.0259414 Accuracy 0.3359375
Step 600 Loss 1.8739252 Accuracy 0.3984375
Step 800 Loss 1.7458391 Accuracy 0.3828125
Step 1000 Loss 1.5310565 Accuracy 0.5078125
Step 1200 Loss 1.5678394 Accuracy 0.546875
Step 1400 Loss 1.3880244 Accuracy 0.578125
Step 1600 Loss 1.4698452 Accuracy 0.59375
Step 1800 Loss 1.2147222 Accuracy 0.640625
Step 2000 Loss 1.13466 Accuracy 0.671875
Step 2200 Loss 1.2738938 Accuracy 0.6015625
Step 2400 Loss 1.3868501 Accuracy 0.625
Step 2600 Loss 1.1675098 Accuracy 0.6484375
Step 2800 Loss 1.1389513 Accuracy 0.6484375
Step 3000 Loss 1.0409479 Accuracy 0.734375
Step 3200 Loss 1.037925 Accuracy 0.734375
Step 3400 Loss 1.0029551 Accuracy 0.6796875
Step 3600 Loss 1.0937593 Accuracy 0.65625
Step 3800 Loss 0.88958937 Accuracy 0.75
Step 4000 Loss 0.8417424 Accuracy 0.671875
Step 4200 Loss 0.9366852 Accuracy 0.7109375
Step 4400 Loss 0.97476625 Accuracy 0.65625
Step 4600 Loss 0.7539169